In [1]:
!nvidia-smi

Thu Dec 22 17:25:30 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:03:00.0 Off |                  Off |
| N/A   44C    P0    39W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install tokenizers

In [3]:
!pip install transformers tinydb

In [4]:
!pip install pytorch_lightning==1.2.10 

In [5]:
!pip install torchtext==0.10.0

ERROR: Could not find a version that satisfies the requirement torchtext==0.10.0 (from versions: 0.1.1, 0.2.0, 0.2.1, 0.2.3, 0.3.1, 0.4.0, 0.5.0, 0.6.0, 0.12.0, 0.13.0, 0.13.1, 0.14.0, 0.14.1)
ERROR: No matching distribution found for torchtext==0.10.0


In [6]:
!pip install gdown --upgrade

In [7]:
!gdown https://drive.google.com/uc?id=1AGtoZDyHNm8FyGqwrIFijAE4_CarvwrT

Downloading...
From: https://drive.google.com/uc?id=1AGtoZDyHNm8FyGqwrIFijAE4_CarvwrT
To: /home/user/pictogram_prediction_pt/pictogram_sentences_final_final.json
100%|██████████████████████████████████████| 8.27M/8.27M [00:00<00:00, 54.5MB/s]


In [8]:
import json
corpus = json.load(open("./pictogram_sentences_final_final.json","r"))

In [9]:
id_sentences = []
ids_ = []
for sentence in corpus:
  s = []
  for t in sentence['pictograms']:
    if t["id"] is None:
      s.append(t["caption"])
      ids_.append(t["caption"])
    else:
      ids_.append(str(t["id"]))
      s.append(str(t["id"]))
  if len(s) >= 3:
    id_sentences.append(" ".join(s))

In [10]:
ids_ = list(set(ids_))
len(ids_)

3675

In [11]:
CORPUS_PATH = "./corpus.txt"
f = open(CORPUS_PATH,"w")
for s in id_sentences:
  f.write(s+"\n")
f.close()

## Criar tokenizer

In [12]:
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.pre_tokenizers import WhitespaceSplit
from tokenizers.processors import BertProcessing

special_tokens = ["[UNK]","[SEP]", "[CLS]", "[PAD]", "[MASK]"]

sense_tokenizer = Tokenizer(WordLevel(unk_token="[UNK]"))
sense_tokenizer.add_special_tokens(special_tokens)
sense_tokenizer.pre_tokenizer = WhitespaceSplit()

sep_token = "[SEP]"
cls_token = "[CLS]"
pad_token = "[PAD]"
unk_token = "[UNK]"
sep_token_id = sense_tokenizer.token_to_id(str(sep_token))
cls_token_id = sense_tokenizer.token_to_id(str(cls_token))
pad_token_id = sense_tokenizer.token_to_id(str(pad_token))
unk_token_id = sense_tokenizer.token_to_id(str(unk_token))

sense_tokenizer.post_processor = BertProcessing(
                (str(sep_token), sep_token_id), (str(cls_token), cls_token_id)
            )

In [13]:
from tokenizers.trainers import WordLevelTrainer
g = WordLevelTrainer(vocab_size = len(ids_)+5,special_tokens=special_tokens,show_progress=True)
sense_tokenizer.train(files=[CORPUS_PATH],trainer=g)

In [14]:
TOKENIZER_PATH = "tokenizer.json"
sense_tokenizer.save(TOKENIZER_PATH)

In [15]:
tokenizer_tokens = set(list(sense_tokenizer.get_vocab().keys()))
len(tokenizer_tokens)

3668

## Ajustar a matriz de embeddings do BERTimbau.

In [16]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, BertForMaskedLM
BASE_MODEL = "neuralmind/bert-base-portuguese-cased"
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
model = BertForMaskedLM.from_pretrained(BASE_MODEL,output_hidden_states=True)
finetune_model = BertForMaskedLM.from_pretrained(BASE_MODEL)

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializi

In [17]:
from transformers import PreTrainedTokenizerFast

cs_tokenizer = PreTrainedTokenizerFast(tokenizer_file=TOKENIZER_PATH)
cs_tokenizer.pad_token = "[PAD]"
cs_tokenizer.sep_token = "[SEP]"
cs_tokenizer.mask_token = "[MASK]"
cs_tokenizer.cls_token = "[CLS]"
cs_tokenizer.unk_token = "[UNK]"

In [18]:
from torch import tensor
# Get weights of the old wte
old_wgts = finetune_model.get_input_embeddings().weight.clone().detach()
print(old_wgts.size())
# Get the mean embedding vetor of the old wte
wgts_m = old_wgts.mean(0)

# Initialize vocab size and weights of the new wte
new_vocab_size = len(cs_tokenizer.vocab)
bert_embeddings = model.get_input_embeddings()

new_wgts = old_wgts.new_zeros(new_vocab_size,old_wgts.size(1))
new_wgts.size()

vectors_m = old_wgts.mean(0)

torch.Size([29794, 768])


In [19]:
!gdown https://drive.google.com/uc?id=1n9LRtmKx-uTTIkCWHLgeTpPB_-bp7cn4

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Downloading...
From: https://drive.google.com/uc?id=1n9LRtmKx-uTTIkCWHLgeTpPB_-bp7cn4
To: /home/user/pictogram_prediction_pt/pictograms.json
100%|██████████████████████████████████████| 7.78M/7.78M [00:00<00:00, 50.5MB/s]


In [20]:
from tinydb import TinyDB,Query
import re
Pictogram = Query()
Keyword = Query()
db = TinyDB("pictograms.json")
picograms = db.table("pictogram")

def get_by_id(id_):
  pics = picograms.search(Pictogram._id == int(id_))
  if len(pics) > 0:
    return pics[0]
  return None

In [98]:
# methods = input_mean | clsn
method = "mean1"

def get_definition_emb_output(sentence,layers):
  tokenized = tokenizer(sentence,return_tensors="pt")
  # tokenized = tokenized.to(device)
  outputs = model(**tokenized)
  hidden_states = torch.stack(list(outputs['hidden_states'])[-layers:]).sum(0)
  return hidden_states[0,0,:].detach().cpu()

def get_definition_emb_input_mean(sentence):
  ids = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentence))
  v_ = bert_embeddings(torch.tensor(ids)).mean(0)
  return v_

def get_definition_emb_mean_output(sentence,layers):
  tokenized = tokenizer(sentence,return_tensors="pt")
  # tokenized = tokenized.to(device)
  outputs = model(**tokenized)
  hidden_states = torch.stack(list(outputs['hidden_states'])[-layers:]).sum(0)
  # return hidden_states
  return hidden_states[0,:,:].detach().cpu().mean(0)

def get_definition_emb(sentence):
  if method == "input_mean":
    return get_definition_emb_input_mean(sentence)
  elif method.startswith("cls"):
    l = int(method[3:])
    return get_definition_emb_output(sentence,l)
  elif method.startswith("mean"):
    l = int(method[4:])
    return get_definition_emb_mean_output(sentence,l)


In [99]:
import torch 
from tqdm import tqdm



for w,i in tqdm(cs_tokenizer.get_vocab().items()):
  tokens = []
  if w.isnumeric():
    pictogram = get_by_id(w)
    if pictogram is not None:
      # tokens = [k['keyword'] for k in pictogram['keywords']]
      for k in pictogram['keywords']:
        if "meaning" in k:
        # print(k['meaning'])
          try:
            tokens.append(k['meaning'].encode('cp1252').decode('utf-8'))
          except:
            tokens.append(k['meaning'])
        try:
          tokens.append(k['keyword'].encode('cp1252').decode('utf-8'))
        except:
          tokens.append(k['keyword'])
    else:
      tokens = [w]  
  else:
    tokens = [w]
  
  ws = " ".join(tokens)
  # ids = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(ws))
  # v_ = bert_embeddings(torch.tensor(ids)).mean(0)
  v_ = get_definition_emb(ws)
  new_wgts[i] = v_

100%|██████████| 3668/3668 [13:06<00:00,  4.66it/s]


In [100]:
from torch import nn

new_wte = nn.Embedding(new_vocab_size,old_wgts.size(1)) # new embeddings
new_wte.weight.data.normal_(mean=0,std=finetune_model.config.initializer_range) 
new_wte.weight.data = new_wgts

finetune_model.resize_token_embeddings(len(cs_tokenizer))
finetune_model.set_input_embeddings(new_wte)
finetune_model.set_output_embeddings(new_wte)

In [101]:
# finetune_model.save_pretrained('/content/gdrive/MyDrive/Doutorado/Corpus construction/execution/files/adjusted-bertimbau-base-14jul')
BASE_MODEL = './def_'+method
finetune_model.save_pretrained(BASE_MODEL)

## Treinamento

In [102]:
import os
os.environ['TRANSFORMERS_NO_ADVISORY_WARNINGS'] = 'true'

In [103]:
from transformers import PreTrainedTokenizerFast
TOKENIZER_PATH = "tokenizer.json"
cs_tokenizer = PreTrainedTokenizerFast(tokenizer_file=TOKENIZER_PATH)
cs_tokenizer.pad_token = "[PAD]"
cs_tokenizer.sep_token = "[SEP]"
cs_tokenizer.mask_token = "[MASK]"
cs_tokenizer.cls_token = "[CLS]"
cs_tokenizer.unk_token = "[UNK]"

In [104]:
f = open("corpus.txt","r")
id_sentences = f.readlines()
id_sentences = [l.rstrip() for l in id_sentences]
id_sentences[0]

'31670 36392 31408 7074 7031 próprio 3218'

In [105]:
# BASE_MODEL = './modelo'
BATCH_SIZE = 768
NUM_WORKERS = 2
MAX_EPOCHS = 200
WARMUP_STEPS = int(MAX_EPOCHS * 0.1)

LEARNING_RATE = 0.00001


LOGS_PATH = "def_logs_{0}".format(method)
LOGGER_INFO = method
LOGGER_VERSION = "1e5"
CHECKPOINTS_PATH = "def_checkpoints_{0}".format(method)
GPUS = 1
ACCUMULATE_GRAD_BATCHES = 4

In [106]:
from sklearn.model_selection import train_test_split
TEST_SIZE = 0.33

train_idx, val_idx = train_test_split(list(range(len(id_sentences))), test_size=TEST_SIZE, random_state=32)
test_idx, val_idx = train_test_split(val_idx, test_size=0.5, random_state=3)

In [107]:
import numpy as np
train_examples = np.array(id_sentences).take(train_idx)
val_examples = np.array(id_sentences).take(val_idx)
test_examples = np.array(id_sentences).take(test_idx)
# test_examples = human_sentences

len(train_examples), len(val_examples),len(test_examples)

(8329, 2052, 2051)

In [108]:
max_len = 13

def tokenize_function(tokenizer,examples):
    # Remove empty lines
    examples = [line for line in examples if len(line) > 0 and not line.isspace()]
    bert = tokenizer(
        examples,
        padding="max_length",
        max_length=max_len,
        return_special_tokens_mask=True,
        truncation=True
    )
    return bert

val_tokenized_examples = tokenize_function(cs_tokenizer,val_examples)
test_tokenized_examples = tokenize_function(cs_tokenizer,test_examples)
train_tokenized_examples = tokenize_function(cs_tokenizer,train_examples)

In [109]:
from torch.utils.data import Dataset, Subset
from torch import tensor

class MyDataset(Dataset):
  def __init__(self, examples):

    self.input_ids = examples['input_ids']
    self.attention_mask = examples['attention_mask']
    self.special_tokens_mask = examples['special_tokens_mask']
  
  def __len__(self):
    return len(self.input_ids)
  
  def __getitem__(self, idx):
    input_ids = tensor(self.input_ids[idx])
    attention_mask = tensor(self.attention_mask[idx])
    special_tokens_mask = tensor(self.special_tokens_mask[idx])

    return {
      "input_ids":input_ids, 
      "attention_mask":attention_mask, 
      "special_tokens_mask":special_tokens_mask
    }

train_dataset = MyDataset(train_tokenized_examples)
test_dataset = MyDataset(test_tokenized_examples)
val_dataset = MyDataset(val_tokenized_examples)


In [110]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=cs_tokenizer, mlm_probability=0.15)
data_collator2 = DataCollatorForLanguageModeling(tokenizer=cs_tokenizer,mlm=False)

In [111]:
def top_n_data_collator(examples):
  
  batch = {
      "input_ids" : torch.stack([example['input_ids'] for example in examples]),
      "attention_mask": torch.stack([example['attention_mask'] for example in examples]),
  }
  special_tokens_mask = torch.stack([example['special_tokens_mask'] for example in examples])
  masked_indices = special_tokens_mask.bool()

  mask_labels = special_tokens_mask.clone()
  probability_matrix = mask_labels
  probability_matrix.masked_fill_(torch.tensor(special_tokens_mask, dtype=torch.bool), value=0.0)
  masked_indices = probability_matrix.bool()

  labels = batch['input_ids'].clone()
  
  for i, row in enumerate(special_tokens_mask.bool()):
    # print(torch.where(row==False)[0])
    mask_id = torch.where(row==False)[0][-2]
    masked_indices[i][mask_id] = True
    batch['input_ids'][i][mask_id] = cs_tokenizer.mask_token_id


  labels[~masked_indices] =  -100
  
  batch['labels'] = labels

  return batch

# top_n_data_collator(train_dataset)

In [112]:
from torch.utils.data import DataLoader
from torch.utils.data.distributed import DistributedSampler

train_dataloader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=True,
    collate_fn=data_collator,
    # drop_last = True,
    shuffle=True
)

val_dataloader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=True,
    collate_fn=data_collator,
    # drop_last = True
)

test_dataloader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    collate_fn=data_collator2,
    pin_memory=True,
)

test_dataloader_topn = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    collate_fn=top_n_data_collator,
    pin_memory=True,
)

In [113]:
from argparse import ArgumentParser
import math
import torch
import torch.nn as nn
import pytorch_lightning as pl
from pytorch_lightning.metrics.functional import accuracy
from pytorch_lightning.callbacks import ModelCheckpoint
from transformers import get_polynomial_decay_schedule_with_warmup
from transformers import AdamW
import torch.nn.functional as F

# BertForMaskedLM.from_pretrained("neuralmind/bert-base-portuguese-cased")

from transformers import BertLMHeadModel
from transformers import BertForMaskedLM


class LitBertClassifier(pl.LightningModule):
    def __init__(self, pretrained_model_name='bert-large-uncased'):
        super().__init__()
        self.save_hyperparameters()
        self.batch_size = BATCH_SIZE
        self.lr = LEARNING_RATE
        self.train_dataset = train_dataset
        self.bert = BertForMaskedLM.from_pretrained(pretrained_model_name)
        self.topns =[1,9,18,25,36]
      
    
    def freeze_to(self, layers):
      for param in self.bert.bert.encoder.layer[:layers].parameters():
        param.requires_grad = False


    def forward(self, input_ids, attention_mask, labels=None):
        if labels == None:
            return self.bert(
                input_ids=input_ids,
                attention_mask=attention_mask,
            )    
        return self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels = labels
        )

    def training_step(self, batch, batch_idx):
        outputs = self._shared_step(batch, batch_idx)
        loss = outputs[0]

        self.log("train_loss", loss, on_epoch=True, prog_bar=True,)

        return loss

    # def training_epoch_end(self, outputs):
    #   #print(outputs)
    #   train_loss = torch.stack([x['loss'] for x in outputs]).mean()
    #   self.log("train_loss", train_loss, on_epoch=True, prog_bar=True,)

    def train_dataloader(self):
      return DataLoader(self.train_dataset,batch_size=self.batch_size,num_workers=NUM_WORKERS,pin_memory=True,collate_fn=data_collator)
      # return DataLoader(self.train_dataset, batch_size=self.batch_size)

    def get_accuracy(self, batch, results):
        y_true = tensor([]).to(torch.device("cuda:0"))
        y_pred = tensor([]).to(torch.device("cuda:0"))
        for idx, ipids in enumerate(batch["input_ids"]):

            idxs = (ipids == tokenizer.mask_token_id).nonzero()
            if idxs.size()[0] > 0:
                device = "cuda:0"
                y_true = torch.cat((y_true, batch['labels'][idx][idxs].resize(1, idxs.size()[0])[0]))
                # y_true = y_true + batch['labels'][idx][idxs].resize(1, idxs.size()[0])[0].tolist()

                idxs_2 = tensor([a[0] for a in idxs])
                idxs_2 = idxs_2.to(device)
                res = torch.index_select(results[idx],0,idxs_2).argmax(1)

                y_pred = torch.cat((y_pred,res))
        if len(y_pred) == 0:
            return None
        return accuracy(tensor(y_true),tensor(y_pred))

    def validation_step(self, batch, batch_idx):
        with torch.no_grad():
          result = self._shared_step(batch, batch_idx)
          loss = result[0].detach()
          # return {"loss":loss}
          # perplexity = torch.exp(loss)
          # acc = self.get_accuracy(batch,result[1].clone())
          # self.log("val_ppl", perplexity, on_epoch=True, prog_bar=True,)
          # self.log("val_acc", acc, on_epoch=True, prog_bar=True,)
          # self.log("val_loss", loss, on_epoch=True, prog_bar=True,)

          return {
              "val_loss":loss
          }
    
    def validation_epoch_end(self, outputs):
        val_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        self.log("val_loss", val_loss, on_epoch=True, prog_bar=True,)
    
    
    def test_step(self, batch, batch_idx):
        with torch.no_grad():
          result = self._shared_step(batch, batch_idx)
          loss = result[0].detach()
          perplexity = torch.exp(loss)
          self.log("test_ppl", perplexity, on_epoch=True, prog_bar=True,)
          self.log("test_loss", loss, on_epoch=True, prog_bar=True,)

          if cs_tokenizer.mask_token_id in batch["input_ids"][0]:

            predictions = F.softmax(result[1], dim=-1)
            labels = batch['labels']
            masked = batch['input_ids']
            n = masked.detach().cpu().numpy()
            predicted = predictions.detach().cpu().numpy()[n == cs_tokenizer.mask_token_id]

            ids = np.argsort(-1*predicted,axis=1)
            for first in self.topns:
              count = 0
              for i, data in enumerate(ids):
                if labels[masked == cs_tokenizer.mask_token_id][i].item() in data[:first]:
                  count += 1
              isin = count/predicted.shape[0]

              self.log("test_"+str(first), isin, on_epoch=True, prog_bar=True,)

          return {
              "test_ppl":perplexity,
              "test_loss":loss
          }
    

    def _shared_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]

        outputs = self.forward(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        return outputs

    def configure_optimizers(self):
      # optimizer =  AdamWeightDecayOptimizer(self.parameters(), lr=LEARNING_RATE,betas=(0.9, 0.999), eps=1e-6)
      # scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=LEARNING_RATE, steps_per_epoch=1735, epochs=500,anneal_strategy='linear')
      # return [optimizer],[scheduler]
      optimizer = AdamW(self.parameters(), lr=self.lr,betas=(0.9, 0.999), eps=1e-08, weight_decay=0.01)
      scheduler = {
          'scheduler': get_polynomial_decay_schedule_with_warmup(optimizer, num_warmup_steps=WARMUP_STEPS,num_training_steps=MAX_EPOCHS,lr_end=1e-08),
          # 'scheduler': get_linear_schedule_with_warmup(optimizer,num_warmup_steps=WARMUP_STEPS,num_training_steps=MAX_EPOCHS,last_epoch=-1),
          'name': 'lr'
      }
      return [optimizer],[scheduler]
    
    def backward(self, loss, optimizer, idx):
        loss.backward()
    


In [114]:
from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning.callbacks import LearningRateMonitor

tb_logger = pl_loggers.TensorBoardLogger(LOGS_PATH,name=LOGGER_INFO, version=LOGGER_VERSION)
lr_monitor = LearningRateMonitor(logging_interval='epoch')

In [115]:

checkpoint_callback = ModelCheckpoint(
    dirpath=CHECKPOINTS_PATH,
    filename='bertimbau-base-{epoch:02d}-{val_loss:.2f}',
    mode='min',
    save_last=True
)

In [116]:
trainer = pl.Trainer(
    accelerator='ddp',
    max_epochs=MAX_EPOCHS,
    logger=tb_logger,
    gpus=GPUS,
    callbacks=[checkpoint_callback,lr_monitor],
    precision=16,
    auto_scale_batch_size="binsearch"
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
Using native 16bit precision.


In [117]:
bert = LitBertClassifier(BASE_MODEL)

In [118]:
# trainer.tune(bert)

In [ ]:
trainer.fit(bert,train_dataloader,val_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type            | Params
-----------------------------------------
0 | bert | BertForMaskedLM | 88.9 M
-----------------------------------------
88.9 M    Trainable params
0         Non-trainable params
88.9 M    Total params
355.454   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Training: 0it [00:00, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Validating: 0it [00:00, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Validating: 0it [00:00, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
trainer.test(bert,test_dataloaders=test_dataloader)

In [ ]:
trainer.test(bert,test_dataloaders=test_dataloader_topn)

In [ ]:
checkpoint_name="./def_checkpoints_input_mean/bertimbau-base-epoch=199-val_loss=2.37.ckpt"
checkpoint = LitBertClassifier.load_from_checkpoint(checkpoint_name)

In [ ]:
trainer.test(checkpoint,test_dataloaders=test_dataloader)

In [ ]:
trainer.test(checkpoint,test_dataloaders=test_dataloader_topn)

In [ ]:
!cp def_checkpoints.zip /content/gdrive/MyDrive/Doutorado/Corpus\ construction/execution/files/def_checkpoints.zip

In [ ]:
!cp def_logs.zip /content/gdrive/MyDrive/Doutorado/Corpus\ construction/execution/files/def_logs.zip